In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
import functools
from my_load_data import load_dataset_fn
#OPTIONAL FOR DEBUGGING
#import import_ipynb

In [2]:
#Parameters of photo
total_num_batches = 124
img_size = 256
num_channels = 3
#Hyper parameters for network
num_classes = 5
batch_size = 10
total_steps = 30
learning_rate = 1e-4
"""Config variable named params, used for forwarding parameters through
whole model"""
#Params used as config file
params = {"learning_rate": learning_rate,
          "img_size": img_size,
          "num_channels":num_channels,
          "num_classes":num_classes,
          "batch_size":batch_size,
          "total_batches":total_num_batches,
          "total_steps":total_steps}

#### Following code serves for nice and smart defyning model as reusable code

In [3]:
def doublewrap(function):
    """
    A decorator of decorator, allowing use of lazy property if no arguments are provided
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope = None, *args, **kwargs):
    """
    Lazy decorator, optimizes code by loading class Model parts only once to memory
    Also its groups tf.Graph, in tensorboard into smaller, more readable parts
    """    
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            #Sorting Graph by Var_scope
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
                print("Initialized Model.{}".format(name))
        return getattr(self, attribute)
    
    return decorator

def accuracy(predictions, labels):
    #Not working, figure out how to make it work for not hot max code
    return (100.0 * np.sum(tf.argmax(predictions, 1) == tf.argmax(labels, 1))
        / predictions.shape[0])

def feed_dict(step, train=True):
    #Used to get input data for network, make it so it can take train or test as input
    #and return proper dataset and label
    if train == True:
        return "error"
    
def save_restore

##### Defining model as reusable function

In [4]:
class Model:
    """
    
    """
    def __init__(self, data, labels, params, mode=None):
        self.data = data
        self.target = labels
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.mode = mode
        self.prediction
        self.optimize
        self.loss
        
    @define_scope
    def prediction(self):
        """
        Main body of neural network, takes data and labels as input,
        returns label prediction for given data(tensor shape same as input)
        """
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels])
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = input_layer, 
                                 filters = 16,
                                 kernel_size = 5,
                                 strides = 1,
                                 padding = "same",
                                 activation = tf.nn.relu)
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")

        #Resizing of pool1 output
        pool_1_flat = tf.reshape(pool1,[-1,int(self.img_size * self.img_size * 16 / 4)])

        #First Dense layer
        dense = tf.layers.dense(inputs = pool_1_flat,
                                units = 128,
                                activation = tf.nn.relu)
        #Second Dense layer
        dense2 = tf.layers.dense(inputs = dense,
                                 units = 64,
                                 activation = tf.nn.relu)
        #Conversion to num_classes output size
        logits = tf.layers.dense(inputs = dense2,
                                 units = self.num_classes,
                                 activation = tf.nn.relu)




        # Softmax output of the neural network
        y_pred = tf.nn.softmax(logits=logits)
        #Optional hotmax output, obsolete, used in classifier
        #y_pred_cls = tf.argmax(y_pred, axis=1)
        return y_pred
    
    
    
    @define_scope
    def optimize(self):
        """
        Optimizer of network, call after model.prediction to optimize network
        """
        cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target, logits=self.prediction)
        optimizer = tf.train.AdamOptimizer(learning_rate = params["learning_rate"])
        return optimizer.minimize(cross_entropy)

    @define_scope
    def loss(self):
        """
        Function returning loss
        """
        with tf.name_scope('bad_predictions'):
            mistakes = tf.not_equal(
                tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
        with tf.name_scope('loss'):
            loss = tf.reduce_mean(tf.cast(mistakes, tf.float32))
            tf.summary.scalar = ('loss', loss)
        return loss

In [5]:
def main(params, mode = None):
    total_steps = params["total_steps"]
    graph = tf.Graph()
    with graph.as_default():
        
        #Placeholders for dataset and labels
        with tf.name_scope('Image'):
            image = tf.placeholder(tf.float32, [None, params["img_size"], params["img_size"], params["num_channels"]])
        with tf.name_scope('Label'):
            label = tf.placeholder(tf.float32, [None, params["num_classes"]])
        
        #Initialization of Model
        model = Model(image, label, params, mode = None)
        print("Starting Session")
        saver = tf.train.Saver()
        with tf.Session(graph=graph) as sess:

            #Initialization of variables
            sess.run(tf.global_variables_initializer())
            print('Variables initialized')

            #Creating save for model session
            save_path=saver.save(sess,"my_net/save_net.ckpt")
            print("Session saved to path:",save_path)

            #Creating log save
            print("Logs save to /results/logs/")
            print("Starting Training")
            #Training
            for step in range(1,total_steps+1):
                dataset, labels = load_dataset_fn(np.round(step/total_steps*params["total_batches"])+1)
                feed_dict = {image: dataset, label: labels}
                pred, _, loss = sess.run([model.prediction, model.optimize,
                                             model.loss], feed_dict= feed_dict)
                #Printing data every 10th step
                if (step % 10 == 0):
                    print('Loss at step %d: %f' % (step, loss))
                    print("Accuracy: {0:2.2f}".format(accuracy(pred, label)))
            
            print("Finnished session")
            writer = tf.summary.FileWriter("./results", sess.graph)
            writer.add_graph(tf.get_default_graph())
            writer.close()
            print("Graph added, work finnished")



In [6]:
tf.reset_default_graph()
if __name__ == '__main__':
     main(params)

Initialized Model.prediction
Initialized Model.optimize
Initialized Model.loss
Starting Session
Variables initialized
Session saved to path: my_net/save_net.ckpt
Logs save to /results/logs/
Starting Training
Loss at step 10: 0.075000
Accuracy: 0.00
Loss at step 20: 0.050000
Accuracy: 0.00
Loss at step 30: 0.100000
Accuracy: 0.00
Finnished session
Graph added, work finnished


In [7]:
###Paste into conda prompt
#   tensorboard --logdir="results/"